# RNN Based molucule generation 

Laurent Cetinsoy

In the hands-on we will train a character based RNN to generate smile molecules

We want to feed smile representations of molecules to an RNN.
The basic idea is we will train it to predict the next smile token of a molecule given the previous one. 

For instance for the following molecule "CC(=O)NC1=CC=C(O)C=C1" will may give to the model

X = "CC(=O)N" 
y = C

and ask the RNN to learn to predict y given X

Like a standard language model ! 

Dowload the following dataset : https://github.com/joeymach/Leveraging-VAE-to-generate-molecules

In [1]:
#!wget https://raw.githubusercontent.com/joeymach/Leveraging-VAE-to-generate-molecules/master/250k_smiles.csv

Import pandas and load the first 1000 lines

In [2]:
import pandas as pd
import numpy as np

In [3]:
!ls data/

250k_smiles.csv  solubility.train.sdf


Display the first rows of the dataframe

In [4]:
df = pd.read_csv("data/250k_smiles.csv")
df.head()

,smiles,logP,qed,SAS
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n,5.05060,0.702012,2.084095
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n,3.11370,0.928975,3.432004
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,4.96778,0.599682,2.470633
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,4.00022,0.690944,2.822753
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...,3.60956,0.789027,4.035182


## Processing the data

We need to do the following things : 

- convert smile tokens to numbers
- build  smile token sequences and corresponding labels pairs

Compute the biggest smile molecule size

In [5]:
df["smiles"][df["smiles"].apply(len).idxmax()]

'C[C@]12C=CC(=O)C=C1CC[C@@H]1[C@@H]2CC[C@@]2(C)[C@H]1CC[C@H]2O[C@@H]1O[C@H](C(=O)[O-])[C@@H](O)[C@H](O)[C@H]1O\n'


Code a function **unic_characters(string)** which return the unic characters in a string


In [6]:
def unic_characters(string):
    return list(set(string))

unic_characters('C[C@]12C=CC(=O)C=C1CC[C@@H]1[C@@H]2CC[C@@]2(C)[C@H]1CC[C@H]2O[C@@H]1O[C@H](C(=O)[O-])[C@@H](O)[C@H](O)[C@H]1O\n')

['@', '-', ']', '[', 'C', '\n', '2', ')', 'H', '=', 'O', '1', '(']

Concatenate all smile string of the pandas dataframe and use **unic_characters** to get the unic_characters 

In [7]:
df_unic_characters = unic_characters(df["smiles"].str.cat(sep=''))
df_unic_characters

['3',
 '-',
 ']',
 '[',
 ')',
 'S',
 'n',
 '1',
 '4',
 '@',
 '8',
 '+',
 '\n',
 '7',
 'B',
 '#',
 's',
 '\\',
 'N',
 'I',
 'o',
 'P',
 '2',
 '(',
 'c',
 '/',
 'C',
 '6',
 'F',
 'l',
 'H',
 '=',
 '5',
 'r',
 'O']

Code a function **map_char_to_int(unic_chars)** which returns a dictionnary where each char is assigned an int value. 
Add a character to specify the end of the molecule (like "\n")


In [8]:
def map_char_to_int(unic_chars):
    return dict((c, i) for i, c in enumerate(unic_chars))
        
    
    #return dict([(uc, ord(uc)) for uc in unic_chars])

map_char_to_int(df_unic_characters)

{'3': 0,
 '-': 1,
 ']': 2,
 '[': 3,
 ')': 4,
 'S': 5,
 'n': 6,
 '1': 7,
 '4': 8,
 '@': 9,
 '8': 10,
 '+': 11,
 '\n': 12,
 '7': 13,
 'B': 14,
 '#': 15,
 's': 16,
 '\\': 17,
 'N': 18,
 'I': 19,
 'o': 20,
 'P': 21,
 '2': 22,
 '(': 23,
 'c': 24,
 '/': 25,
 'C': 26,
 '6': 27,
 'F': 28,
 'l': 29,
 'H': 30,
 '=': 31,
 '5': 32,
 'r': 33,
 'O': 34}

Code a function map_int_to_char(unic_chars) which returns the reverse mapping. 

If you want you can merge both functions in a class

In [9]:
def map_int_to_char(unic_chars):
    return dict((i, c) for i, c in enumerate(unic_chars))

map_int_to_char(df_unic_characters)

{0: '3',
 1: '-',
 2: ']',
 3: '[',
 4: ')',
 5: 'S',
 6: 'n',
 7: '1',
 8: '4',
 9: '@',
 10: '8',
 11: '+',
 12: '\n',
 13: '7',
 14: 'B',
 15: '#',
 16: 's',
 17: '\\',
 18: 'N',
 19: 'I',
 20: 'o',
 21: 'P',
 22: '2',
 23: '(',
 24: 'c',
 25: '/',
 26: 'C',
 27: '6',
 28: 'F',
 29: 'l',
 30: 'H',
 31: '=',
 32: '5',
 33: 'r',
 34: 'O'}

For each smile molecule add the ending token to it

In [10]:
df["smiles"][249454]

'O=C(CC(c1ccccc1)c1ccccc1)N1CCN(S(=O)(=O)c2ccccc2[N+](=O)[O-])CC1\n'

In [11]:
df["smiles"].where(df["smiles"].str[-1] != '\n').any()

False

## Building the dataset

Now we will create the dataset so that it has the good share for our Keras LSTM model

Remember Keras recurrent models expect a 3D array with shapes (n_examples, seq_len, n_features)



What will be n_features in our case ? 

Code a function **build_X_and_y(string, i_char, seq_lenght)** which takes a string, a **seq_length** number and a position. 


It should create X by by getting all character between i and i + seq_length 
and create y by getting the character following the X sequence
it returns X and y

Test your function on the following string "" with seq_length = 4 and i = [1, 2, 3]

In [12]:
def build_X_and_y(string, i_char, seq_lenght):
    
    X = [string[i: i + seq_lenght] for i in i_char]
    y = [string[i + seq_lenght] if i + seq_lenght < len(string) else "" for i in i_char]
    
    return X, y

In [13]:
build_X_and_y("azertyazeaze", [1,2,3], 4)

(['zert', 'erty', 'rtya'], ['y', 'a', 'z'])

By using build_X_and_y and map_char_to_int build a list nameed X_train and a list named y_train 

In [14]:
df["smiles"][0]

'CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n'

In [15]:
df["smiles"].apply(len).min()

10

In [16]:
def get_max_X_and_y_from_str(string, seq_length=5):
    return build_X_and_y(string, range(len(string) - seq_length), seq_length)

In [17]:
dict_char_to_int = map_char_to_int(df_unic_characters)
dict_char_to_int

{'3': 0,
 '-': 1,
 ']': 2,
 '[': 3,
 ')': 4,
 'S': 5,
 'n': 6,
 '1': 7,
 '4': 8,
 '@': 9,
 '8': 10,
 '+': 11,
 '\n': 12,
 '7': 13,
 'B': 14,
 '#': 15,
 's': 16,
 '\\': 17,
 'N': 18,
 'I': 19,
 'o': 20,
 'P': 21,
 '2': 22,
 '(': 23,
 'c': 24,
 '/': 25,
 'C': 26,
 '6': 27,
 'F': 28,
 'l': 29,
 'H': 30,
 '=': 31,
 '5': 32,
 'r': 33,
 'O': 34}

In [18]:
df["smiles_encoded"] = df["smiles"].apply(lambda string: [dict_char_to_int[c] for c in string])

In [19]:
lists = df["smiles_encoded"].apply(get_max_X_and_y_from_str)

In [20]:
lists.shape

(249455,)

In [21]:
lists

0         ([[26, 26, 23, 26, 4], [26, 23, 26, 4, 23], [2...
1         ([[26, 3, 26, 9, 9], [3, 26, 9, 9, 30], [26, 9...
2         ([[18, 15, 26, 24, 7], [15, 26, 24, 7, 24], [2...
3         ([[26, 26, 34, 26, 23], [26, 34, 26, 23, 31], ...
4         ([[18, 15, 26, 26, 7], [15, 26, 26, 7, 31], [2...
                                ...                        
249450    ([[26, 26, 7, 23, 26], [26, 7, 23, 26, 4], [7,...
249451    ([[26, 6, 7, 24, 24], [6, 7, 24, 24, 6], [7, 2...
249452    ([[26, 24, 7, 24, 24], [24, 7, 24, 24, 24], [7...
249453    ([[26, 24, 7, 24, 24], [24, 7, 24, 24, 23], [7...
249454    ([[34, 31, 26, 23, 26], [31, 26, 23, 26, 26], ...
Name: smiles_encoded, Length: 249455, dtype: object

In [22]:
X_train = lists.apply(lambda x: x[0])
y_train = lists.apply(lambda x: x[1])

In [23]:
X_train.shape

(249455,)

In [24]:
# FIXME instead of for loop we could use numpy ufunc
X_train = [e for li in X_train for e in li]
y_train = [e for li in y_train for e in li]

Create numpy arrays from the lists

In [25]:
import numpy as np

X_train, y_train = np.array(X_train), np.array(y_train)

Reshape the X numpy array (n_examples, seq_lenght, 1)

In [26]:
X_train.shape

(10055646, 5)

In [27]:
X_train = X_train.reshape(-1, 5, 1)
X_train.shape

(10055646, 5, 1)

Normalize X by dividing each values by the total number of unic characters

In [31]:
X_train = X_train / len(dict_char_to_int)

Import Keras and build (at least) a two layered LSTM network with 128 neurone in each.

You can also add Dropoutlayers

Do you think you should use the return_sequences = True ? If yes, when ? 


Add a Dense layer on top with with the appropriate activation function and number of neurones 


In [73]:
from keras.models import Sequential
from keras.layers import LSTM, Flatten, Dense

model = Sequential()

model.add(LSTM(128, return_sequences=True, activation="tanh", input_shape=(5, 1)))
model.add(LSTM(128, activation="tanh"))

model.add(Flatten())

model.add(Dense(1024, activation="relu"))
model.add(Dense(1, activation="linear"))

model = Sequential()

model.add(LSTM(30, activation="tanh", input_shape=(None, 1)))

model.add(Flatten())

model.add(Dense(800, activation="relu"))
model.add(Dense(len(dict_char_to_int), activation="softmax"))

Compile the model with the appropriate loss function and the adam optimizer

In [74]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

Train the model on 20 epochs and 10 examples (yeah you read correctly) and check that the model overfits ! 

In [75]:
X_train[:10].shape

(10, 5, 1)

In [76]:
X_small = X_train[:10]
y_small = y_train[:10]

In [80]:
model.fit(X_train, y_train, epochs=20)

Epoch 1/20
  2515/314239 [..............................] - ETA: 25:14 - loss: 2.7532 - accuracy: 0.2119

KeyboardInterrupt: 

If it does not overfit try to fix data prep and model architecture so it does

Create a function **make_prediction(seed_start)** which takes a starting string sequence and uses it to generate a molecule


generate a molecule of your overfitted model

Make a model checkpoint so that the model is saved after each epoch
if you train on a plateform and it stops you do not lose your training 

Now go to your favorite plateform (colab or something else) and train the dataset on the whole data for 10 epochs and batch size 256 

it should take a long time so either follow the class or go take a nap 

Generate between 100 and 1000 molecules. 

create a list where molecules have between 10 and 50 atoms

With rdkit compute the Quantified Estimated Drug likelyness (QED) of each molecule in this subset

Bonus 1 : predict the molecule solubility of your generated molecules 

Bonus 2 : try to adapt a transformer model training from hugging face to see if it is better